In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

<h1 align="center"> Data Compilation </h1>
<h4 align="center"> Christine Zhang </h4>

---

In [1]:
import numpy as np
import pandas as pd

In [2]:
mut_pd = pd.read_csv('../Seashore-ludlow/v22.anno.ccl_mut_features.txt', sep ="\t")
mut = mut_pd.as_matrix()
mut_pd.head(n=10)

,mutation_call,hugo_gene_symbol,cell_line_feature,index_ccl
0,CCLE Hybrid Capture Mutations,AAK1,AAK1_p._ANY_CODING_MUTATION,2;3;4;6;7;8;10;12;13;15;16;18;19;20;22;23;24;2...
1,CCLE Hybrid Capture Mutations,AAK1,AAK1_p.Q541,2;3;4;6;7;8;10;12;13;15;16;18;19;20;22;23;24;2...
2,CCLE Hybrid Capture Mutations,AAK1,AAK1_p.Q541indel2>Q,2;3;4;6;7;8;10;12;13;15;16;18;19;20;22;23;24;2...
3,CCLE Hybrid Capture Mutations,AATK,AATK_p._ANY_CODING_MUTATION,38;60;72;114;126;145;149;157;183;207;226;269;3...
4,CCLE Hybrid Capture Mutations,ABCA3,ABCA3_p._ANY_CODING_MUTATION,8;36;52;67;113;126;127;128;130;131;132;133;135...
5,CCLE Hybrid Capture Mutations,ABCA3,ABCA3_p.L654,36;113;171;209
6,CCLE Hybrid Capture Mutations,ABCA3,ABCA3_p.L654V,36;113;171;209
7,CCLE Hybrid Capture Mutations,ABCA3,ABCA3_p.N124,127;343;556
8,CCLE Hybrid Capture Mutations,ABCA3,ABCA3_p.N124S,127;343;556
9,CCLE Hybrid Capture Mutations,ABCC3,ABCC3_p._ANY_CODING_MUTATION,17;66;128;131;133;135;157;168;182;193;198;202;...


In [3]:
len(mut_pd["hugo_gene_symbol"].drop_duplicates())

1574

In [4]:
index_pd = pd.read_csv('../Seashore-ludlow/v22.meta.per_cell_line.txt', sep ="\t")
correlation_pd = index_pd[["index_ccl", "master_ccl_id"]]
correlation = correlation_pd.as_matrix()
correlation_pd.head(n=10)

,index_ccl,master_ccl_id
0,1,144
1,2,475
2,3,528
3,4,571
4,5,572
5,6,678
6,7,720
7,8,852
8,9,1030
9,10,1051


In [5]:
# reformat array by ccl
mut_by_ccl = []
for row in range(0, len(mut)):
    for each_mut in mut[row][-1].split(";"):
        mut_by_ccl.append([int(each_mut), mut[row][0], mut[row][1], mut[row][2]])
mut_by_ccl_sorted = sorted(mut_by_ccl)
mut_sorted = np.array(mut_by_ccl_sorted)

# covert index ccl to master ccl id
corr_lst = list(correlation[:,0])
for mut_index, row in enumerate(mut_sorted):
    if int(row[0]) in correlation[:,0]:
        corr_index = corr_lst.index(int(row[0]))
        mut_sorted[mut_index, 0] = correlation[corr_index, 1]
print mut_sorted

[['144' 'CCLE Hybrid Capture Mutations' 'AKAP9' 'AKAP9_p.X1335']
 ['144' 'CCLE Hybrid Capture Mutations' 'AKAP9' 'AKAP9_p.X1335ins']
 ['144' 'CCLE Hybrid Capture Mutations' 'AKAP9' 'AKAP9_p.X1335insQ']
 ..., 
 ['1250' 'CCLE Oncomap (v3) Mutations' 'TP53' 'TP53_p.R175']
 ['1250' 'CCLE Oncomap (v3) Mutations' 'TP53' 'TP53_p.R175H']
 ['1250' 'CCLE Oncomap (v3) Mutations' 'TP53' 'TP53_p._ANY_CODING_MUTATION']]


In [6]:
copynum_pd = pd.read_csv('../Rees/v21.data.cnv_avg_log2.txt', skiprows = [], sep ="\t")
copynum = copynum_pd.as_matrix()
copynum_pd.head(n=10)

,master_ccl_id,entrez_gene_id,copy_number_var_avg_log2
0,1,1,0.0661
1,1,503538,0.0661
2,1,29974,-0.0080
3,1,2,-0.0233
4,1,144571,-0.0233
5,1,144568,-0.0233
6,1,3,-0.0233
7,1,53947,0.0229
8,1,51146,-0.0459
9,1,100506677,-0.0143


In [7]:
master_ccl = copynum_pd["entrez_gene_id"].drop_duplicates()
len(master_ccl)

23174

In [8]:
auc_pd = pd.read_csv('dataset1.csv', skiprows = [0,1], sep =",", 
                          names = ["Cclid", "Cellline", "Sites", "Histology", "Subhistology", "Culture", "SNP", "AUC"])
auc = auc_pd.as_matrix()
auc_pd.head(n=10)

,Cclid,Cellline,Sites,Histology,Subhistology,Culture,SNP,AUC
0,3,5637,urinary_tract,carcinoma,NS,RPMI001,SNP-matched-reference,2.473
1,7,22RV1,prostate,carcinoma,NS,RPMI001,SNP-matched-reference,2.142
2,10,42MGBA,central_nervous_system,glioma,astrocytoma_Grade_IV,RPMIMEM001,SNP-matched-reference,3.380
3,15,647V,urinary_tract,carcinoma,transitional_cell_carcinoma,DMEM011,SNP-matched-reference,3.374
4,16,769P,kidney,carcinoma,clear_cell_renal_cell_carcinoma,RPMI001,SNP-matched-reference,3.123
5,19,786O,kidney,carcinoma,clear_cell_renal_cell_carcinoma,RPMI001,SNP-matched-reference,3.779
6,21,8505C,thyroid,carcinoma,anaplastic_carcinoma,EMEM001,SNP-matched-reference,3.471
7,22,8MGBA,central_nervous_system,glioma,astrocytoma_Grade_IV,EMEM005,SNP-matched-reference,2.625
8,23,A101D,skin,malignant_melanoma,NS,DMEM001,SNP-matched-reference,4.677
9,25,A172,central_nervous_system,glioma,astrocytoma_Grade_IV,DMEM001,SNP-matched-reference,2.751


In [9]:
def intersect (first, second, combined, toggle, type_var):
    if toggle == 1:
        first = list(set(first[:,0]))
        second = list(set(second[:,0]))
    for row in first:
        if type_var == "str":
            if str(row) in second:
                combined.append(row)
        if type_var == "float":
            if float(row) in second:
                combined.append(row)
        if type_var == "int":
            if int(row) in second:
                combined.append(row)
    return combined, len(combined)

In [10]:
auc_mut, mut_copynum, auc_mut_copynum = [], [], []

auc_mut_lst, auc_mut_len = intersect (auc, mut_sorted, auc_mut, 1, "str")
mut_copynum_lst, mut_copynum_len = intersect (auc, copynum, mut_copynum, 1, "float")
all_overlap, all_len = intersect (auc_mut_lst, mut_copynum_lst, auc_mut_copynum, 0, "int")

print "Total Number of Master CCL Ids: " + str(all_len)

Total Number of Master CCL Ids: 472
